In [1]:
import numpy as np
import pandas as pd
import re

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
import umap

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import kneighbors_graph

import pythainlp
from pythainlp.corpus.common import thai_words
from pythainlp.util import Trie
import collections

In [2]:
df=pd.read_csv('7204.csv')

In [3]:
df.head(10)

,comment,label
0,อีตัวเงินตัวทอง,1
1,น่าสงสาร สัมภเวสี พวกเศษขยะอย่างพวกมึง,1
2,ไม่ชั่วทำไม่ได้นะ,1
3,ปากบอกไม่เป็นไรๆแต่ในใจโกรธและเสียหน้ามาก นางม...,1
4,คำว่า ผู้ดี กับ ไพร่,0
5,ดูได้จากปฏิกริยา ความคิด กมลสันดาน และการกระทำ,0
6,ขอบคุณป้าที่ทำให้เห็นธาตุแท้ของคนบางคนยิ่งขึ้น,0
7,เขารู้ว่าเป็นคนเลวเขาจึงไม่ให้ถ่ายไง,0
8,จิตใจคิดร้ายหวังให้เขาเสื่อมเสีย,0
9,คุณป้าท่านสุภาพเป็นผู้ดีพอพูดจาไพเราะขอไห้ขายด...,0


In [4]:
df.label.value_counts()

0    576
1    374
9      1
Name: label, dtype: int64

In [5]:
# clean data

In [6]:
special_characters = "/[!@#$%^&*']/g"

specialchar_pattern = re.compile(special_characters)

In [7]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

In [8]:
number_pattern = re.compile("[0-9]")
space_pattern = re.compile("\s+")
dot_pattern = re.compile(r"\.+")
backslash_pattern = re.compile(r"\\+")

In [9]:
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = ['u', 'b', 'n', 'nn', 'nn-', '\n']
screening_words = stopwords + removed_words

new_words = {'สามกีบ','มารศาสนา'}

words = new_words.union(thai_words())

custom_dictionary_trie = Trie(words)

def tokenize_to_list(sentence):
  merged = []
  words = pythainlp.word_tokenize(str(sentence), engine='newmm', custom_dict=custom_dictionary_trie)
  for word in words:
    if word not in screening_words:
      merged.append(word)
  return merged

In [10]:
def final(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm', custom_dict=custom_dictionary_trie)
  for word in words:
    if word not in screening_words:
      merged=merged+word
  return merged

In [11]:
df

,comment,label
0,อีตัวเงินตัวทอง,1
1,น่าสงสาร สัมภเวสี พวกเศษขยะอย่างพวกมึง,1
2,ไม่ชั่วทำไม่ได้นะ,1
3,ปากบอกไม่เป็นไรๆแต่ในใจโกรธและเสียหน้ามาก นางม...,1
4,คำว่า ผู้ดี กับ ไพร่,0
...,...,...
946,ยุคนี้มันเสรีแล้ว รักใครก็ได้ขอให้รักกันก็พอ,0
947,ความรักคือความเข้าใจกัน,0
948,ยิ่งเวลาเมานี่พูดสนุกปากเลยแต่ละคน,0
949,น่าอาย ขุดทอง,1


In [12]:
df['comment'] = df['comment'].apply(lambda x: emoji_pattern.sub(r'', x))
df['comment'] = df['comment'].apply(lambda x: specialchar_pattern.sub(r'', x))
df['comment'] = df['comment'].apply(lambda x: number_pattern.sub(r'', x))
df['comment'] = df['comment'].apply(lambda x: space_pattern.sub(r'', x))
df['comment'] = df['comment'].apply(lambda x: dot_pattern.sub(r'', x))
df['comment'] = df['comment'].apply(lambda x: backslash_pattern.sub(r'', x))
df['texts_tokenized'] = df['comment'].apply(lambda x: tokenize_to_list(x))
df['final']=df['comment'].apply(lambda x: final(x))

In [13]:
data=df

In [14]:
data

,comment,label,texts_tokenized,final
0,อีตัวเงินตัวทอง,1,"[อี, ตัวเงินตัวทอง]",อีตัวเงินตัวทอง
1,น่าสงสารสัมภเวสีพวกเศษขยะอย่างพวกมึง,1,"[น่าสงสาร, สัมภเวสี, เศษ, ขยะ]",น่าสงสารสัมภเวสีเศษขยะ
2,ไม่ชั่วทำไม่ได้นะ,1,"[ชั่ว, ทำ]",ชั่วทำ
3,ปากบอกไม่เป็นไรๆแต่ในใจโกรธและเสียหน้ามากนางมี...,1,"[ปาก, ใจ, โกรธ, เสียหน้า, ปม, ใจ, อยู่แล้ว, คน...",ปากใจโกรธเสียหน้าปมใจอยู่แล้วคนต้อนรับโวยวายน่...
4,คำว่าผู้ดีกับไพร่,0,"[ผู้ดี, ไพร่]",ผู้ดีไพร่
...,...,...,...,...
946,ยุคนี้มันเสรีแล้วรักใครก็ได้ขอให้รักกันก็พอ,0,"[ยุค, เสรี, รัก, ใครก็ได้, ขอให้, รัก]",ยุคเสรีรักใครก็ได้ขอให้รัก
947,ความรักคือความเข้าใจกัน,0,"[ความรัก, ความเข้าใจ]",ความรักความเข้าใจ
948,ยิ่งเวลาเมานี่พูดสนุกปากเลยแต่ละคน,0,"[เวลา, เมา, สนุกปาก, แต่ละคน]",เวลาเมาสนุกปากแต่ละคน
949,น่าอายขุดทอง,1,"[น่าอาย, ขุด, ทอง]",น่าอายขุดทอง


In [15]:
from sklearn.model_selection import train_test_split

In [16]:
y=data.label
x=data.final

In [17]:
from gensim import models
from gensim.models import Word2Vec, KeyedVectors

In [18]:
model = Word2Vec(sentences=data.texts_tokenized, vector_size=100, window=5, min_count=5, workers=4, sg=0, epochs=5) 

In [19]:
Xtrain,Xtest,Ytrain,Ytest=train_test_split(x,y,test_size=0.3)

In [20]:
with open('th.tsv') as f:
  vocab = []
  W = None
  vec = ''
  for line in f.readlines():
    temp = line.split('\t')
    if len(temp) == 3:
      vocab.append(temp[1])
      if int(temp[0]) % 1000 == 0:
        print('\r' + temp[0], end='')
      wordvector = np.fromstring(re.sub('\s+', ' ', vec)[1:-1], sep=' ', dtype=np.float32)
      if len(wordvector) > 0:
        if W is None:
          W = wordvector.copy()
        else:
          W = np.vstack((W, wordvector))
      vec = temp[-1]
    elif len(temp) == 1:
      vec += temp[-1]
  W = np.vstack((W, wordvector))
print('\n')
print(len(vocab))
print(W.shape)

0

/Users/macproretina132015/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  # This is added back by InteractiveShellApp.init_path()


30000

30225
(30225, 300)


In [21]:
from attacut import tokenize

In [22]:
def sent2idx(x):
  xidx = []
  for w in tokenize(x):
    if w in vocab:
      xidx.append(vocab.index(w))
  return np.array(xidx)

In [23]:
Xtrain_idx = []
maxlen = 0
for x in Xtrain:
  Xtrain_idx.append(sent2idx(x))
  if len(Xtrain_idx[-1]) > maxlen:
    maxlen = len(Xtrain_idx[-1])
  print(len(Xtrain_idx[-1]))

3
10
2
24
9
4
4
4
1
9
0
9
11
8
8
3
13
3
14
4
6
2
7
5
4
5
4
7
2
2
10
12
4
4
4
8
4
4
6
3
13
4
9
6
9
11
3
5
9
10
8
7
1
8
1
2
15
1
9
5
6
3
4
0
6
9
4
3
7
3
4
2
5
4
12
8
3
9
6
5
7
3
3
6
5
6
1
3
2
10
5
10
0
3
10
16
7
12
2
3
7
4
9
4
2
12
4
2
8
10
2
5
7
2
4
1
2
1
7
3
6
0
2
2
4
4
7
2
18
5
8
2
4
2
5
3
3
4
11
10
4
8
6
5
5
15
2
1
2
7
4
1
4
0
6
9
23
4
0
10
9
5
3
3
6
4
3
10
5
7
2
4
5
4
1
4
5
4
3
14
3
3
6
13
9
4
5
5
1
2
5
3
9
13
5
5
3
5
7
22
2
11
4
5
6
5
8
4
1
1
2
6
5
5
3
6
1
11
3
5
5
8
2
3
1
0
3
7
3
6
12
4
9
5
2
13
6
9
13
2
4
8
0
6
5
1
4
4
4
12
8
5
2
3
3
3
19
2
2
0
3
7
2
5
11
6
6
3
15
14
9
6
2
5
4
2
7
9
0
12
7
4
4
0
19
3
11
4
4
2
1
6
8
8
2
3
3
2
17
6
14
9
0
3
4
10
2
21
5
1
6
3
15
2
1
3
3
13
12
34
6
4
5
2
8
4
5
4
12
23
3
7
3
2
7
16
4
12
2
6
6
5
8
6
1
11
1
11
5
1
11
15
7
3
8
8
9
5
9
7
5
9
4
6
4
5
6
7
9
3
5
13
9
16
9
5
3
12
10
10
3
1
31
2
2
5
5
9
9
2
1
11
11
2
1
12
0
10
7
2
7
4
9
6
7
2
1
2
2
7
3
12
2
11
4
6
6
6
1
1
0
1
3
3
15
5
7
5
7
4
0
5
6
9
1
10
3
1
13
0
3
1
1
2
4
4
6
0
8
6
1
14
1
3
14
8
5
4
9
1
6
5


In [24]:
vocab = [''] + vocab
print(len(vocab))

30226


In [25]:
W = np.vstack((np.random.rand(*W[0].shape), W))

In [26]:
for i, x in enumerate(Xtrain_idx):
  if len(x) < maxlen:
    Xtrain_idx[i] = np.hstack((x, np.zeros(maxlen-len(x))))
Xtrain_idx = np.array(Xtrain_idx)

In [27]:
Xtrain_idx.shape

(665, 34)

In [28]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import re
from tensorflow.keras.layers import Dropout

In [213]:
model = keras.Sequential()
model.add(keras.layers.Embedding(W.shape[0], W.shape[1], name='embed'))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(20, return_sequences=True)))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(8)))
model.add(keras.layers.Dense(30, activation='relu'))
model.add(keras.layers.Dense(2))

model.compile(loss=keras.losses.sparse_categorical_crossentropy, optimizer=keras.optimizers.SGD(learning_rate=1e-4))

In [214]:
model.get_layer('embed').set_weights([W])
model.get_layer('embed').trainable = False

In [215]:
model.fit(Xtrain_idx, Ytrain,epochs=20)

Epoch 1/20
21/21 [==============================] - 8s 41ms/step - loss: 2.4592
Epoch 2/20
21/21 [==============================] - 1s 40ms/step - loss: 1.3946
Epoch 3/20
21/21 [==============================] - 1s 41ms/step - loss: 0.8183
Epoch 4/20
21/21 [==============================] - 1s 40ms/step - loss: 0.7927
Epoch 5/20
21/21 [==============================] - 1s 41ms/step - loss: 0.7796
Epoch 6/20
21/21 [==============================] - 1s 40ms/step - loss: 0.7557
Epoch 7/20
21/21 [==============================] - 1s 39ms/step - loss: 0.7463
Epoch 8/20
21/21 [==============================] - 1s 39ms/step - loss: 0.7421
Epoch 9/20
21/21 [==============================] - 1s 40ms/step - loss: 0.7392
Epoch 10/20
21/21 [==============================] - 1s 40ms/step - loss: 0.7064
Epoch 11/20
21/21 [==============================] - 1s 41ms/step - loss: 0.6977
Epoch 12/20
21/21 [==============================] - 1s 40ms/step - loss: 0.6946
Epoch 13/20
21/21 [==================

In [216]:
Xtest_idx = []
for x in Xtest:
  Xtest_idx.append(sent2idx(x))
  if len(Xtest_idx[-1]) > maxlen:
    Xtest_idx[-1] =  Xtest_idx[-1][:maxlen]
  else:
    Xtest_idx[-1] = np.hstack((Xtest_idx[-1], np.zeros(maxlen-len(Xtest_idx[-1]))))
Xtest_idx = np.array(Xtest_idx)

In [217]:
model.predict(Xtest_idx).argmax(axis=1)

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0])

In [218]:
y_pred=model.predict(Xtest_idx).argmax(axis=1)

In [219]:
from sklearn.metrics import accuracy_score
accuracy_score(Ytest,y_pred)

0.5244755244755245

In [220]:
# predict pyspark

In [221]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.streaming import StreamingContext

In [222]:
spark = SparkSession.builder\
        .master("local")\
        .appName('word count')\
        .getOrCreate()
sc=spark.sparkContext

In [223]:
in_text=sc.textFile('/Users/macproretina132015/Desktop/text.txt')

In [224]:
text=in_text.collect()

In [225]:
text

['อีสายเหลือง',
 'อย่ามั่นหน้าว่าตัวเองสวยเลย',
 'เค้าก็น่ารักนะ',
 'ขุดทองแน่ๆ พวกเกย์ขุดทอง',
 'อีบ้า สติไม่ดี',
 'น่ารักมากกกกกก',
 'นิสัยดีนะ',
 '']

In [226]:
Xfuture_idx = []
for x in text:
  Xfuture_idx.append(sent2idx(x))
  if len(Xfuture_idx[-1]) > maxlen:
    Xfuture_idx[-1] =  Xfuture_idx[-1][:maxlen]
  else:
    Xfuture_idx[-1] = np.hstack((Xfuture_idx[-1], np.zeros(maxlen-len(Xfuture_idx[-1]))))
Xfuture_idx = np.array(Xfuture_idx)

In [227]:
model.predict(Xfuture_idx).argmax(axis=1)

array([1, 1, 0, 0, 0, 0, 0, 0])